In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntSlider, FloatSlider, interact_manual

In [2]:
def plot_simulation_data(selected_simulation=0):
    global global_target_conversion_rate
    
    # Calculate daily mean and standard deviation
    daily_mean = daily_conversion_rates.mean(axis=1)
    
    plt.figure(figsize=(14, 7), dpi=200)
    days_range = np.arange(daily_conversion_rates.shape[0])
    upper_limit = global_target_conversion_rate + 3 * avg_stddev
    lower_limit = global_target_conversion_rate - 3 * avg_stddev
    plt.ylim(lower_limit if lower_limit > 0 else 0, upper_limit)
    
    # Plotting the daily mean conversion rate
    plt.plot(days_range, daily_mean, color='green', linewidth=2, label='Sampled Daily Mean Conversion Rate')
    
    # Plotting the selected simulation
    plt.plot(days_range, daily_conversion_rates[:, selected_simulation], color='red', linestyle='-', linewidth=1.5, label=f'Selected Simulation {selected_simulation}')
    
    # Plotting the target conversion rate
    plt.axhline(y=global_target_conversion_rate, color='blue', linestyle='-', label='Target Conversion Rate')
    
    # Filling the 68% confidence interval (±1 avg_stddev) around theoretical target conversion rate
    plt.fill_between(days_range, global_target_conversion_rate - avg_stddev, global_target_conversion_rate + avg_stddev, color='blue', alpha=0.075, label='68% Confidence Interval')
    
    # Filling the 95% confidence interval (±2 avg_stddev) around theoretical target conversion rate
    plt.fill_between(days_range, global_target_conversion_rate - 2*avg_stddev, global_target_conversion_rate + 2*avg_stddev, color='blue', alpha=0.075, label='95% Confidence Interval')
    
    plt.title("Daily Conversion Rates Simulation")
    plt.xlabel("Day")
    plt.ylabel("Conversion Rate")
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.legend(loc='upper right')
    plt.show()

In [3]:
# generate_data function
def generate_data(num_simulations=100, visitors_per_day=1000, target_conversion_rate_param=0.051, total_days=60, seed=None):
    # Global variables
    global daily_conversion_rates, daily_stddev_rates, avg_stddev, global_target_conversion_rate
    
    # Simulate data
    visitor_data_array = np.zeros((visitors_per_day, total_days, num_simulations))
    np.random.seed(seed)
    visitor_data_array = np.random.rand(visitors_per_day, total_days, num_simulations)
    visitor_data_array = (visitor_data_array <= target_conversion_rate_param).astype(int)
    daily_conversion_rates = visitor_data_array.sum(axis=0) / visitors_per_day
    daily_stddev_rates = daily_conversion_rates.std(axis=1)
    global_target_conversion_rate = target_conversion_rate_param
    avg_stddev = daily_stddev_rates.mean()  

    # Display interactive plot
    interactive_plot = interactive(plot_simulation_data, selected_simulation=(0, num_simulations-1, 1))
    display(interactive_plot)
 

In [4]:
# UI to set parameters and run simulations using interact_manual
interact_manual(generate_data, 
                num_simulations=IntSlider(min=1, max=1000, value=100, description='Simulations', continuous_update=False),
                visitors_per_day=IntSlider(min=1, max=100000, value=1000, description='Visitors/Day', continuous_update=False),
                target_conversion_rate_param=FloatSlider(min=0.001, max=1.000, value=0.051, step=0.001, description='Conversion Rate', continuous_update=False),
                total_days=IntSlider(min=2, max=365, value=60, description='Total Days', continuous_update=False),
                seed=IntSlider(min=0, max=1000, value=None, description='Random Seed', continuous_update=False, optional=True));

interactive(children=(IntSlider(value=100, continuous_update=False, description='Simulations', max=1000, min=1…